In [1]:
from ultralytics import YOLO
import cv2
import os

# YOLO modelini yükle
model = YOLO("termal_model.pt")  # kendi .pt model yolunu yaz

# Sınıf isimleri
class_map = model.names

# Frame klasörü
frame_folder = "termal"
frames = sorted([f for f in os.listdir(frame_folder) if f.endswith('.jpeg')])

for f in frames:
    img_path = os.path.join(frame_folder, f)
    frame = cv2.imread(img_path)

    # YOLO tahmin
    results = model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    preds = results[0].boxes

    # Kutuları çiz
    for box, cls_id in zip(preds.xywh, preds.cls):
        xc, yc, bw, bh = box.tolist()
        x1, y1 = int(xc - bw/2), int(yc - bh/2)
        x2, y2 = int(xc + bw/2), int(yc + bh/2)
        label = class_map.get(int(cls_id), str(int(cls_id)))
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255,140,0), 2)
        cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255,140,0), 2, cv2.LINE_AA)

    # Göster
    cv2.imshow("Termal YOLO Detection", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


KeyboardInterrupt: 

In [15]:
import os
import cv2
import io
import base64
from PIL import Image
from ultralytics import YOLO
from openai import OpenAI
from dotenv import load_dotenv

# 🔐 API anahtarını yükle
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API anahtarı yüklenemedi.")

client = OpenAI(api_key=api_key)

# 🔍 GPT-4O ile silah tespiti
def llm_detect_weapon_fast(image_crop):
    try:
        pil_image = Image.fromarray(cv2.cvtColor(image_crop, cv2.COLOR_BGR2RGB))
        buffered = io.BytesIO()
        pil_image.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode()

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Bu kişide silah var mı? Sadece 'Evet' veya 'Hayır' yaz."},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
                    ]
                }
            ],
            max_tokens=10,
            temperature=0.0
        )
        cevap = response.choices[0].message.content.strip()
        print(f"LLM cevabı: {cevap}")  # 🖨️ Anında terminal çıktısı
        return cevap
    except Exception as e:
        print("LLM hata:", e)
        return "Bilinmiyor"

# 📦 YOLO modeli
model = YOLO("termal_model.pt")
class_map = model.names

# 📷 Görseli yükle
img_path = os.path.join("termal", "FLIR_video_03383.jpeg")
frame = cv2.imread(img_path)

# 🔍 YOLO tahmini
results = model.predict(frame, imgsz=320, conf=0.4, verbose=False)
preds = results[0].boxes

# 📊 Sayaçlar
silahli_sayisi = 0
silahsiz_sayisi = 0

# 🧍‍♂️ Her kişi için işlem yap
for box, cls_id in zip(preds.xywh, preds.cls):
    xc, yc, bw, bh = box.tolist()
    x1, y1 = int(xc - bw / 2), int(yc - bh / 2)
    x2, y2 = int(xc + bw / 2), int(yc + bh / 2)
    label = class_map.get(int(cls_id), str(int(cls_id)))

    if label.lower() == "person":
        person_crop = frame[y1:y2, x1:x2].copy()
        if person_crop.size > 0:
            weapon_result = llm_detect_weapon_fast(person_crop)
            label += f" | Silah: {weapon_result}"

            # Sayaç güncelle
            if "Evet" in weapon_result:
                silahli_sayisi += 1
            elif "Hayır" in weapon_result:
                silahsiz_sayisi += 1

        # Görsel üzerine yaz
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 140, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 140, 0), 2, cv2.LINE_AA)

# 📊 Özet çıktısı
print("\n==========================")
print(f"🔫 Silahlı kişi sayısı   : {silahli_sayisi}")
print(f"🧍‍♂️ Silahsız kişi sayısı : {silahsiz_sayisi}")
print("==========================")

# 🖼️ Görseli göster
cv2.imshow("FLIR Detection - Silah Tespiti", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

LLM cevabı: Üzgünüm, bu isteğe yardımcı ol
LLM cevabı: Üzgünüm, bu görüntüdeki

🔫 Silahlı kişi sayısı   : 0
🧍‍♂️ Silahsız kişi sayısı : 0


In [23]:
import os
import cv2
import io
import base64
from PIL import Image
from ultralytics import YOLO
from openai import OpenAI
from dotenv import load_dotenv

# 🔐 API anahtarını yükle
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API anahtarı yüklenemedi.")

client = OpenAI(api_key=api_key)

# 🔍 GPT-4O ile silah tespiti
def llm_detect_weapon_fast(image_crop):
    try:
        pil_image = Image.fromarray(cv2.cvtColor(image_crop, cv2.COLOR_BGR2RGB))
        buffered = io.BytesIO()
        pil_image.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode()

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Bu kişinin elinde veya üzerinde silah olduğu düşünülebilir mi? Sadece 'Evet' ya da 'Hayır' yaz."},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
                    ]
                }
            ],
            max_tokens=10,
            temperature=0.0
        )
        cevap = response.choices[0].message.content.strip()
        print(f"LLM cevabı: {cevap}")
        return cevap
    except Exception as e:
        print("LLM hata:", e)
        return "Bilinmiyor"

# 📦 YOLO modeli
model = YOLO("termal_model.pt")
class_map = model.names

# 📷 Görseli yükle
img_path = os.path.join("termal", "FLIR_video_03901.jpeg")
frame = cv2.imread(img_path)

# 🔍 YOLO tahmini
results = model.predict(frame, imgsz=320, conf=0.4, verbose=False)
preds = results[0].boxes

# 📊 Sayaçlar
silahli_sayisi = 0
silahsiz_sayisi = 0

# 🧍‍♂️ Her kişi için işlem yap
for box, cls_id in zip(preds.xywh, preds.cls):
    xc, yc, bw, bh = box.tolist()
    x1, y1 = int(xc - bw / 2), int(yc - bh / 2)
    x2, y2 = int(xc + bw / 2), int(yc + bh / 2)
    label = class_map.get(int(cls_id), str(int(cls_id)))

    if label.lower() == "person":
        # 🔧 Yeni kırpma stratejisi: margin + sınır kontrolü
        margin = 10
        x1_adj = max(0, x1 + margin)
        y1_adj = max(0, y1 + margin)
        x2_adj = min(frame.shape[1], x2 - margin)
        y2_adj = min(frame.shape[0], y2 - margin)

        person_crop = frame[y1_adj:y2_adj, x1_adj:x2_adj].copy()

        # 📏 Crop boyut kontrolü
        h, w = person_crop.shape[:2]
        if h < 50 or w < 50 or h * w > 400 * 400:
            print("⚠️ Uygun olmayan boyutta crop atlandı.")
            continue

        # 🔍 LLM analizi
        weapon_result = llm_detect_weapon_fast(person_crop)
        label += f" | Silah: {weapon_result}"

        # Sayaç güncelle
        if "Evet" in weapon_result:
            silahli_sayisi += 1
        elif "Hayır" in weapon_result:
            silahsiz_sayisi += 1

        # Görsel üzerine yaz
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 140, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 140, 0), 2, cv2.LINE_AA)

# 📊 Özet çıktısı
print("\n==========================")
print(f"🔫 Silahlı kişi sayısı   : {silahli_sayisi}")
print(f"🧍‍♂️ Silahsız kişi sayısı : {silahsiz_sayisi}")
print("==========================")

# 🖼️ Görseli göster
cv2.imshow("FLIR Detection - Silah Tespiti", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

LLM cevabı: Hayır.
LLM cevabı: Hayır.

🔫 Silahlı kişi sayısı   : 0
🧍‍♂️ Silahsız kişi sayısı : 2
